## 복습

* 이 tutorial은 자동차 세트로 연습해본 GAN을 좀 더 가벼운 세트로 연습해보자는 뜻으로 만들었습니다. (Acknowledgement: 애플의 Erik Linder-Norén님의 공개 코드를 약간 변형하여 제작되었습니다 )

* MNIST는 클래식한 머신러닝 데이터 셋으로 필기된 숫자들을 회색조로 만든 것으로 28*28 픽셀 이미지를 보고 0부터 9까지를 판별하는 용, 즉 이미지 분류의 타이타닉 같은 역할을 하는 데이터 셋입니다. 이를 GAN으로 해서 생성해 보겠습니다

* 코드의 기본 개념은 거의 같은데 데이터가 다르니 처음 데이터를 불러오고 처리하는 부분은 좀 달리 하겠습니다

* 궁극적으로 훈련 모델과 툴은 후반에 비슷하게 하여 기존에 배운 연습과 비슷하게 합니다

### 텐서플로와 다른 라이브러리 불러오기

*우선 텐서 플로우를 설치합니다. 

In [1]:
# for cpu and gpu
! pip install --upgrade tensorflow

     |████████████████████████████████| 394.7MB 44kB/s 
     |████████████████████████████████| 471kB 46.8MB/s 
     |████████████████████████████████| 10.6MB 50.8MB/s 
     |████████████████████████████████| 14.5MB 46.1MB/s 
     |████████████████████████████████| 3.8MB 52.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: grpcio 1.34.0
    Uninstalling grpcio-1.34.0:
      Successfully uninstalled grpcio-1.34.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
   

In [2]:
%load_ext tensorboard
import os
from glob import glob
import time
import random

import IPython.display as display
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import PIL
from PIL import Image
import imageio
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import Input
from keras.layers import (Activation, BatchNormalization, Concatenate, Dense, Dropout,
                          Embedding, Flatten, Input, multiply, Reshape, ZeroPadding2D)
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers.advanced_activations import LeakyReLU, ReLU
from keras.models import Model, Sequential
from keras.optimizers import Adam
from sklearn.preprocessing import OrdinalEncoder
%matplotlib inline

In [3]:
tf.__version__

'2.4.0'

In [4]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.n_classes = 10
        self.noise_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.make_discriminator_model()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.make_generator_model()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.noise_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines decision
        # and the label of that image
        real = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], real)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def make_generator_model(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.noise_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.noise_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.n_classes, self.noise_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def make_discriminator_model(self):

        model = Sequential()

        model.add(Dense(512, input_dim=np.prod(self.img_shape)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.n_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        decision = model(model_input)

        return Model([img, label], decision)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (train_image, train_label), (_, _) = tf.keras.datasets.mnist.load_data()

        # Configure input
        train_image = (train_image.astype(np.float32) - 127.5) / 127.5
        train_image = np.expand_dims(train_image, axis=3)
        train_label = train_label.reshape(-1, 1)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, train_image.shape[0], batch_size)
            imgs, labels = train_image[idx], train_label[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, labels])

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch([imgs, labels], real)
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Condition on labels
            sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], real)

            # Plot the progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 2, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.arange(0, 10).reshape(-1, 1)

        gen_imgs = self.generator.predict([noise, sampled_labels])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].set_title("Digit: {}".format(sampled_labels[cnt]))
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images{}.png".format(epoch))
        plt.close()


if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=50000, batch_size=32, sample_interval=5000)

Streaming output truncated to the last 5000 lines.
45000 [D loss: 0.689240, acc.: 60.94%] [G loss: 0.772412]
45001 [D loss: 0.692126, acc.: 54.69%] [G loss: 0.752314]
45002 [D loss: 0.637642, acc.: 60.94%] [G loss: 0.812548]
45003 [D loss: 0.639204, acc.: 64.06%] [G loss: 0.779952]
45004 [D loss: 0.685647, acc.: 50.00%] [G loss: 0.777341]
45005 [D loss: 0.683547, acc.: 56.25%] [G loss: 0.821009]
45006 [D loss: 0.638097, acc.: 67.19%] [G loss: 0.830930]
45007 [D loss: 0.723073, acc.: 45.31%] [G loss: 0.796004]
45008 [D loss: 0.727118, acc.: 50.00%] [G loss: 0.790975]
45009 [D loss: 0.640341, acc.: 62.50%] [G loss: 0.779194]
45010 [D loss: 0.694987, acc.: 60.94%] [G loss: 0.818372]
45011 [D loss: 0.646909, acc.: 64.06%] [G loss: 0.777420]
45012 [D loss: 0.660384, acc.: 53.12%] [G loss: 0.778645]
45013 [D loss: 0.697497, acc.: 48.44%] [G loss: 0.893521]
45014 [D loss: 0.704966, acc.: 53.12%] [G loss: 0.822958]
45015 [D loss: 0.664835, acc.: 53.12%] [G loss: 0.780825]
45016 [D loss: 0.6553

## 다음 단계

다음에는 똑 같은 DB로 LSGAN을 해봅니다